In [2]:
import numpy as np
import pandas as pd

In [3]:
from models import *
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 50)

In [4]:
pd.options.plotting.backend = "plotly" 


# Compare Agg Functions best models vs. Last Fold best models

### Agg Functions best models

In [5]:
best_model_p_func = pd.read_csv('data/best_model_p_func.csv').rename({'Unnamed: 0':"func"},axis=1)
best_model_p_func

,func,model,param_config
0,average,XGBClassifier_date_params,32
1,average_invstd_25,DecisionTreeClassifier_date_params,37
2,average_invstd_50,DecisionTreeClassifier_date_params,37
3,average_invstd_75,DecisionTreeClassifier_date_params,37
4,invstdev,DecisionTreeClassifier_date_params,0
5,weighted_avg,XGBClassifier_date_params,35
6,regret_average,XGBClassifier_date_params,32
7,regret_invstd,lgb_LGBMClassifier_date_params,31
8,mean_invstd_regret_25,lgb_LGBMClassifier_date_params,31
9,mean_invstd_regret_50,lgb_LGBMClassifier_date_params,31


In [46]:
outputs_metrics

,Unnamed: 0,threshold,precision_test,precision_threshold,filename,model,param_config,time
0,100,0.555556,0.441926,0.366391,"(0, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier_date_params,0,0
1,450,0.485549,0.447646,0.472284,"(1, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier_date_params,0,1
2,800,0.424198,0.397163,0.497674,"(2, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier_date_params,0,2
3,1150,0.516340,0.439252,0.411141,"(3, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier_date_params,0,3
4,1500,0.666667,0.424479,0.464646,"(4, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier_date_params,0,4
...,...,...,...,...,...,...,...,...
3495,1949,0.635021,0.680672,0.659864,"(5, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier_date_params,9,5
3496,2299,0.638066,0.728997,0.679104,"(6, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier_date_params,9,6
3497,2649,0.648194,0.597260,0.774390,"(7, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier_date_params,9,7
3498,2999,0.657300,0.534274,0.683794,"(8, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier_date_params,9,8


In [48]:
results_ = results

In [68]:
outputs_metrics_ = outputs_metrics
outputs_metrics_

,Unnamed: 0,threshold,precision_test,precision_threshold,filename,model,param_config,time,model_config
0,100,0.555556,0.441926,0.366391,"(0, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier,0,0,DecisionTreeClassifier0
1,450,0.485549,0.447646,0.472284,"(1, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier,0,1,DecisionTreeClassifier0
2,800,0.424198,0.397163,0.497674,"(2, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier,0,2,DecisionTreeClassifier0
3,1150,0.516340,0.439252,0.411141,"(3, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier,0,3,DecisionTreeClassifier0
4,1500,0.666667,0.424479,0.464646,"(4, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier,0,4,DecisionTreeClassifier0
...,...,...,...,...,...,...,...,...,...
3495,1949,0.635021,0.680672,0.659864,"(5, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier,9,5,lgb_LGBMClassifier9
3496,2299,0.638066,0.728997,0.679104,"(6, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier,9,6,lgb_LGBMClassifier9
3497,2649,0.648194,0.597260,0.774390,"(7, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier,9,7,lgb_LGBMClassifier9
3498,2999,0.657300,0.534274,0.683794,"(8, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier,9,8,lgb_LGBMClassifier9


In [64]:
(outputs_metrics.model+outputs_metrics.param_config.apply(str)).isin(results_.model+results_.param_config.apply(str)).sum()

2620

In [72]:
results_['threshold'],results_['precision_threshold'],results_['time'] = np.nan,np.nan,10
results_['model_config'] =results_.model+results_.param_config.apply(str)
results_['precision_test'] =results_['precision_prod']
outputs_metrics_wt_last = pd.concat([outputs_metrics_,results_[['Unnamed: 0','threshold','precision_test','precision_threshold','filename','model','param_config','time','model_config']]]).drop_duplicates()

### Best model last Fold

In [85]:
outputs_metrics_wt_last.query('time == 0').model_config.value_counts().sort_index()

DecisionTreeClassifier0     1
DecisionTreeClassifier10    1
DecisionTreeClassifier11    1
DecisionTreeClassifier16    1
DecisionTreeClassifier18    1
DecisionTreeClassifier2     1
DecisionTreeClassifier20    1
DecisionTreeClassifier23    1
DecisionTreeClassifier24    1
DecisionTreeClassifier25    1
DecisionTreeClassifier26    1
DecisionTreeClassifier28    1
DecisionTreeClassifier29    1
DecisionTreeClassifier33    1
DecisionTreeClassifier34    1
DecisionTreeClassifier35    1
DecisionTreeClassifier37    1
DecisionTreeClassifier38    1
DecisionTreeClassifier39    1
DecisionTreeClassifier4     1
DecisionTreeClassifier40    1
DecisionTreeClassifier41    1
DecisionTreeClassifier44    1
DecisionTreeClassifier45    1
DecisionTreeClassifier47    1
DecisionTreeClassifier6     1
DecisionTreeClassifier8     1
LogisticRegression0         1
LogisticRegression1         1
LogisticRegression10        1
LogisticRegression11        1
LogisticRegression12        1
LogisticRegression13        1
LogisticRe

In [86]:
outputs_metrics_wt_last.query('time == 10').model_config.value_counts().sort_index()

DecisionTreeClassifier0     1
DecisionTreeClassifier10    1
DecisionTreeClassifier11    1
DecisionTreeClassifier16    1
DecisionTreeClassifier18    1
DecisionTreeClassifier2     1
DecisionTreeClassifier20    1
DecisionTreeClassifier23    1
DecisionTreeClassifier24    1
DecisionTreeClassifier25    1
DecisionTreeClassifier26    1
DecisionTreeClassifier28    1
DecisionTreeClassifier29    1
DecisionTreeClassifier33    1
DecisionTreeClassifier34    1
DecisionTreeClassifier35    1
DecisionTreeClassifier37    1
DecisionTreeClassifier38    1
DecisionTreeClassifier39    1
DecisionTreeClassifier4     1
DecisionTreeClassifier40    1
DecisionTreeClassifier41    1
DecisionTreeClassifier44    1
DecisionTreeClassifier45    1
DecisionTreeClassifier47    1
DecisionTreeClassifier6     1
DecisionTreeClassifier8     1
LogisticRegression0         1
LogisticRegression1         1
LogisticRegression10        1
LogisticRegression11        1
LogisticRegression12        1
LogisticRegression13        1
LogisticRe

In [91]:
 outputs_metrics_wt_last.sort_values(['model_config','time']).plot(kind='line',x='time',y='precision_test',color='model', width=1300, height=500,labels={'metric':'precision'})

In [93]:
from PIL import ImageColor
import plotly
fig = outputs_metrics_wt_last.plot(kind='line',x='time',y='precision_test',color='model',line_group="param_config", width=1300, height=500,labels={'metric':'precision'})

for d in fig.data:
#    d.marker.size = 10
    d.line.color = 'rgba('+str(ImageColor.getcolor(d.line.color, "RGB"))[1:-1]+",0.3"+')'

    #'rgba(100,0,255,'+'0.1'+')'
fig.update_layout({"yaxis"+str(i+1): dict(range = [0, 1]) for i in range(7)})
plotly.offline.plot(fig, filename='fig_precision_times.html')
fig

In [60]:
outputs_metrics = pd.read_csv("data/outputs_metrics_refined.csv")
outputs_metrics['model'] = outputs_metrics['model'].str[:-12]
outputs_metrics['model_config'] = outputs_metrics['model'] + outputs_metrics['param_config'].apply(str)

bad_models = outputs_metrics.query('precision_test <0.05')['model_config'].drop_duplicates()
bad_models = list(bad_models)

outputs_metrics = outputs_metrics.query(f'model_config != {bad_models}')


In [61]:

outputs_metrics

,Unnamed: 0,threshold,precision_test,precision_threshold,filename,model,param_config,time,model_config
0,100,0.555556,0.441926,0.366391,"(0, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier,0,0,DecisionTreeClassifier0
1,450,0.485549,0.447646,0.472284,"(1, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier,0,1,DecisionTreeClassifier0
2,800,0.424198,0.397163,0.497674,"(2, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier,0,2,DecisionTreeClassifier0
3,1150,0.516340,0.439252,0.411141,"(3, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier,0,3,DecisionTreeClassifier0
4,1500,0.666667,0.424479,0.464646,"(4, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier,0,4,DecisionTreeClassifier0
...,...,...,...,...,...,...,...,...,...
3495,1949,0.635021,0.680672,0.659864,"(5, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier,9,5,lgb_LGBMClassifier9
3496,2299,0.638066,0.728997,0.679104,"(6, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier,9,6,lgb_LGBMClassifier9
3497,2649,0.648194,0.597260,0.774390,"(7, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier,9,7,lgb_LGBMClassifier9
3498,2999,0.657300,0.534274,0.683794,"(8, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier,9,8,lgb_LGBMClassifier9


In [11]:
 outputs_metrics.query(f"model =='{best_model_last[0]}' & param_config == {best_model_last[1]}")

NameError: name 'best_model_last' is not defined

### Ranks on the Production set

In [12]:
file = 'data/donors_choose_prepared.csv'
data = pd.read_csv(file)
data['as_of_date'] = pd.to_datetime(data.as_of_date)
start_test_production = data.as_of_date.max() - pd.DateOffset(months=2)
start_thresh_prod = data.as_of_date.max() - pd.DateOffset(months=3)
start_train_prod = data.as_of_date.max() - pd.DateOffset(months=6)
data

,entity_id,as_of_date,project_features_entity_id_all_grade_level_Grades 3-5_sum,project_features_entity_id_all_grade_level_Grades 6-8_sum,project_features_entity_id_all_grade_level_Grades 9-12_sum,project_features_entity_id_all_grade_level_Grades PreK-2_sum,project_features_entity_id_all_grade_level__NULL_sum,project_features_entity_id_all_poverty_level__NULL_sum,project_features_entity_id_all_poverty_level_high pov_sum,project_features_entity_id_all_poverty_level_highest _sum,...,donation_features_entity_id_all_teacher_funding_rate_2yr_imp,donation_features_entity_id_all_zip_avg_donations_1yr_sum,donation_features_entity_id_all_zip_avg_donations_1yr_imp,donation_features_entity_id_all_zip_avg_donations_2yr_sum,donation_features_entity_id_all_zip_avg_donations_2yr_imp,donation_features_entity_id_all_zip_funding_rate_1yr_sum,donation_features_entity_id_all_zip_funding_rate_1yr_imp,donation_features_entity_id_all_zip_funding_rate_2yr_sum,donation_features_entity_id_all_zip_funding_rate_2yr_imp,quickstart_label
0,333897,2011-09-01,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0000,1.0,0.00000,1.0,0.000000,1.0,0.000000,1.0,1.0
1,333592,2011-09-01,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0000,1.0,0.00000,1.0,0.000000,1.0,0.000000,1.0,0.0
2,333593,2011-09-01,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0000,1.0,0.00000,1.0,0.000000,1.0,0.000000,1.0,0.0
3,333594,2011-09-01,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0000,1.0,0.00000,1.0,0.000000,1.0,0.000000,1.0,1.0
4,333595,2011-09-01,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0000,1.0,0.00000,1.0,0.000000,1.0,0.000000,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165875,168451,2013-01-31,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,651.6060,0.0,642.58154,0.0,1.000000,0.0,1.000000,0.0,0.0
165876,168452,2013-01-31,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,517.6779,0.0,557.33350,0.0,0.584906,0.0,0.704918,0.0,1.0
165877,168453,2013-01-31,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0000,1.0,0.00000,1.0,0.000000,1.0,0.000000,1.0,1.0
165878,168430,2013-01-31,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,260.5326,0.0,304.09012,0.0,0.583333,0.0,0.679245,0.0,0.0


In [13]:
data.as_of_date.max() - pd.DateOffset(months=3)

Timestamp('2012-10-31 00:00:00')

In [14]:
data.as_of_date.max()

Timestamp('2013-01-31 00:00:00')

In [15]:
 #data.query(f'as_of_date > "{start_train_prod}" & as_of_date <= "{start_thresh_prod}" ').drop(['entity_id','as_of_date','quickstart_label'],axis=1)

In [16]:
#pd.read_csv('data/time_windows_frames.csv')

In [17]:
import re

X_train_prod = data.query(f'as_of_date > "{start_train_prod}" & as_of_date <= "{start_thresh_prod}" ')\
            .drop(['entity_id','as_of_date','quickstart_label'],axis=1)\
            .applymap(float)\
            .rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
y_train_prod = data.query(f'as_of_date > "{start_train_prod}" & as_of_date <= "{start_thresh_prod}" ')\
        ['quickstart_label'].apply(int)

X_test_prod = data.query(f'as_of_date > "{start_thresh_prod}" & as_of_date <= "{start_test_production}" ')\
    .drop(['entity_id','as_of_date','quickstart_label'],axis=1).applymap(float)\
        .rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
y_test_prod = data.query(f'as_of_date > "{start_thresh_prod}" & as_of_date <= "{start_test_production}" ')\
        ['quickstart_label'].apply(int)

X_val_prod = data.query(f'as_of_date > "{start_test_production}"  ')\
        .drop(['entity_id','as_of_date','quickstart_label'],axis=1).applymap(float)\
        .rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
y_val_prod = data.query(f'as_of_date > "{start_test_production}" ')\
        ['quickstart_label'].apply(int)

In [18]:
len(X_train_prod)/165880

0.23997468049192186

In [19]:
outputs_metrics = pd.read_csv("data/outputs_metrics_refined.csv")
outputs_metrics

,Unnamed: 0,threshold,precision_test,precision_threshold,filename,model,param_config,time
0,100,0.555556,0.441926,0.366391,"(0, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier_date_params,0,0
1,450,0.485549,0.447646,0.472284,"(1, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier_date_params,0,1
2,800,0.424198,0.397163,0.497674,"(2, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier_date_params,0,2
3,1150,0.516340,0.439252,0.411141,"(3, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier_date_params,0,3
4,1500,0.666667,0.424479,0.464646,"(4, 2, 0)_DecisionTreeClassifier_date_params.csv",DecisionTreeClassifier_date_params,0,4
...,...,...,...,...,...,...,...,...
3495,1949,0.635021,0.680672,0.659864,"(5, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier_date_params,9,5
3496,2299,0.638066,0.728997,0.679104,"(6, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier_date_params,9,6
3497,2649,0.648194,0.597260,0.774390,"(7, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier_date_params,9,7
3498,2999,0.657300,0.534274,0.683794,"(8, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier_date_params,9,8


In [20]:
all_model_configs = pd.read_csv("data/outputs_metrics_refined.csv")[['filename','model','param_config']].groupby(['model','param_config']).first().reset_index()
all_model_configs.sort_values('filename')

,model,param_config,filename
150,RandomForestClassifier_date_params,0,"(0, 0, 0)_RandomForestClassifier_date_params.csv"
151,RandomForestClassifier_date_params,1,"(0, 0, 1)_RandomForestClassifier_date_params.csv"
160,RandomForestClassifier_date_params,10,"(0, 0, 10)_RandomForestClassifier_date_params.csv"
161,RandomForestClassifier_date_params,11,"(0, 0, 11)_RandomForestClassifier_date_params.csv"
162,RandomForestClassifier_date_params,12,"(0, 0, 12)_RandomForestClassifier_date_params.csv"
163,RandomForestClassifier_date_params,13,"(0, 0, 13)_RandomForestClassifier_date_params.csv"
164,RandomForestClassifier_date_params,14,"(0, 0, 14)_RandomForestClassifier_date_params.csv"
165,RandomForestClassifier_date_params,15,"(0, 0, 15)_RandomForestClassifier_date_params.csv"
166,RandomForestClassifier_date_params,16,"(0, 0, 16)_RandomForestClassifier_date_params.csv"
167,RandomForestClassifier_date_params,17,"(0, 0, 17)_RandomForestClassifier_date_params.csv"


In [21]:
find_threshold = find_threshold_2

In [22]:
#results = []
important_models = all_model_configs.loc[(all_model_configs['model']+all_model_configs['param_config'].apply(str)).isin(best_model_p_func['model']+best_model_p_func['param_config'].apply(str)).to_frame('is_important').query('is_important == True').index]

for _,(model,param_config,filename) in all_model_configs.iterrows():
    print((model,param_config,filename))
    model = model[:-12]
    if not (model+str(param_config)  in np.array(results, dtype=np.object)[:,0]+np.array(results, dtype=np.str)[:,1].astype(np.object)):
        
        print((model,param_config,filename))
        model_params = pd.read_csv('outputs_models3/'+filename,index_col = 'Unnamed: 0')['0'].get_p

        model_params = model_params.replace("nan", "None")
        best_model_config = clfs_names_dict[model](**eval(model_params))

        best_model_config.fit(X_train_prod,y_train_prod)
        y_proba_val_prod = best_model_config.predict_proba(X_val_prod)[:,1]
        threshold_prod,precision_val_prod = find_threshold(y_val_prod,y_proba_val_prod)
        y_proba_test_prod = best_model_config.predict_proba(X_test_prod)[:,1]
        precision_prod = precision_score(y_test_prod,y_proba_test_prod>threshold_prod)
        results.append([model,param_config,filename,precision_prod,model_params])
        display(pd.DataFrame(results).iloc[-1])

('DecisionTreeClassifier_date_params', 0, '(0, 2, 0)_DecisionTreeClassifier_date_params.csv')


NameError: name 'results' is not defined

In [ ]:
all_model_configs.sample(5)

In [27]:
#results.to_csv('production_results.csv')
#results = pd.read_csv('production_results.csv')

In [28]:

results = pd.DataFrame(results,columns = ['model','param_config','filename','precision_prod','params'])
results['rank_prod']= results.precision_prod.rank(ascending = False)
results

,model,param_config,filename,precision_prod,params,rank_prod
0,DecisionTreeClassifier,0,"(0, 2, 0)_DecisionTreeClassifier_date_params.csv",0.435897,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",244.0
1,DecisionTreeClassifier,1,"(0, 2, 1)_DecisionTreeClassifier_date_params.csv",0.000000,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",316.0
2,DecisionTreeClassifier,2,"(0, 2, 2)_DecisionTreeClassifier_date_params.csv",0.386228,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",255.0
3,DecisionTreeClassifier,3,"(0, 2, 3)_DecisionTreeClassifier_date_params.csv",0.000000,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",316.0
4,DecisionTreeClassifier,4,"(0, 2, 4)_DecisionTreeClassifier_date_params.csv",0.429066,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",247.0
5,DecisionTreeClassifier,5,"(0, 2, 5)_DecisionTreeClassifier_date_params.csv",0.408377,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",252.0
6,DecisionTreeClassifier,6,"(0, 2, 6)_DecisionTreeClassifier_date_params.csv",0.471154,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",218.0
7,DecisionTreeClassifier,7,"(0, 2, 7)_DecisionTreeClassifier_date_params.csv",0.000000,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",316.0
8,DecisionTreeClassifier,8,"(0, 2, 8)_DecisionTreeClassifier_date_params.csv",0.463320,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",223.0
9,DecisionTreeClassifier,9,"(0, 2, 9)_DecisionTreeClassifier_date_params.csv",0.000000,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",316.0


In [29]:

results_important = pd.DataFrame(results,columns = ['model','param_config','filename','precision_prod'])
results_important['rank_prod']= results_important.precision_prod.rank(ascending = False)
results_important

,model,param_config,filename,precision_prod,rank_prod
0,DecisionTreeClassifier,0,"(0, 2, 0)_DecisionTreeClassifier_date_params.csv",0.435897,244.0
1,DecisionTreeClassifier,1,"(0, 2, 1)_DecisionTreeClassifier_date_params.csv",0.000000,316.0
2,DecisionTreeClassifier,2,"(0, 2, 2)_DecisionTreeClassifier_date_params.csv",0.386228,255.0
3,DecisionTreeClassifier,3,"(0, 2, 3)_DecisionTreeClassifier_date_params.csv",0.000000,316.0
4,DecisionTreeClassifier,4,"(0, 2, 4)_DecisionTreeClassifier_date_params.csv",0.429066,247.0
5,DecisionTreeClassifier,5,"(0, 2, 5)_DecisionTreeClassifier_date_params.csv",0.408377,252.0
6,DecisionTreeClassifier,6,"(0, 2, 6)_DecisionTreeClassifier_date_params.csv",0.471154,218.0
7,DecisionTreeClassifier,7,"(0, 2, 7)_DecisionTreeClassifier_date_params.csv",0.000000,316.0
8,DecisionTreeClassifier,8,"(0, 2, 8)_DecisionTreeClassifier_date_params.csv",0.463320,223.0
9,DecisionTreeClassifier,9,"(0, 2, 9)_DecisionTreeClassifier_date_params.csv",0.000000,316.0


In [30]:
best_model_p_func['model'] = best_model_p_func['model'].str[:-12] 

NameError: name 'best_model_last' is not defined

In [33]:
best_model_last_info = outputs_metrics.query("time == 9").sort_values('precision_test',ascending=False).iloc[1]
best_model_last = list(best_model_last_info[['model','param_config']].values)
best_model_last_precision = best_model_last_info.precision_test
best_model_last[0] = best_model_last[0][:-12]
best_model_last

['lgb_LGBMClassifier', 9]

In [34]:
#best_model_p_func = best_model_p_func.iloc[:-2]
best_model_p_func = best_model_p_func.append(\
                            pd.DataFrame([['best_model_last_fold_2nd'] + best_model_last]\
                                         ,columns =best_model_p_func.columns ),ignore_index = True)
#best_model_p_func = best_model_p_func.iloc[:-2]
best_model_p_func

,func,model,param_config
0,average,XGBClassifier,32
1,average_invstd_25,DecisionTreeClassifier,37
2,average_invstd_50,DecisionTreeClassifier,37
3,average_invstd_75,DecisionTreeClassifier,37
4,invstdev,DecisionTreeClassifier,0
5,weighted_avg,XGBClassifier,35
6,regret_average,XGBClassifier,32
7,regret_invstd,lgb_LGBMClassifier,31
8,mean_invstd_regret_25,lgb_LGBMClassifier,31
9,mean_invstd_regret_50,lgb_LGBMClassifier,31


In [35]:
results = pd.read_csv('production_results.csv')
results['model_config'] = results['model'] + results['param_config'].apply(str)

results = results.query(f'model_config != {bad_models}')
results['rank_prod'] = results['rank_prod'].rank()
results.sort_values('rank_prod')

,Unnamed: 0,model,param_config,filename,precision_prod,params,rank_prod,model_config
272,272,XGBClassifier,22,"(0, 4, 22)_XGBClassifier_date_params.csv",0.720207,"{'objective': 'binary:logistic', 'base_score':...",1.0,XGBClassifier22
293,293,XGBClassifier,43,"(0, 4, 43)_XGBClassifier_date_params.csv",0.694444,"{'objective': 'binary:logistic', 'base_score':...",2.0,XGBClassifier43
181,181,RandomForestClassifier,31,"(0, 0, 31)_RandomForestClassifier_date_params.csv",0.686869,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",3.0,RandomForestClassifier31
166,166,RandomForestClassifier,16,"(0, 0, 16)_RandomForestClassifier_date_params.csv",0.685990,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",4.0,RandomForestClassifier16
268,268,XGBClassifier,18,"(0, 4, 18)_XGBClassifier_date_params.csv",0.676692,"{'objective': 'binary:logistic', 'base_score':...",5.0,XGBClassifier18
295,295,XGBClassifier,45,"(0, 4, 45)_XGBClassifier_date_params.csv",0.674157,"{'objective': 'binary:logistic', 'base_score':...",6.0,XGBClassifier45
154,154,RandomForestClassifier,4,"(0, 0, 4)_RandomForestClassifier_date_params.csv",0.673575,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",7.0,RandomForestClassifier4
346,346,lgb_LGBMClassifier,46,"(0, 3, 46)_lgb_LGBMClassifier_date_params.csv",0.672956,"{'boosting_type': 'dart', 'class_weight': None...",8.0,lgb_LGBMClassifier46
302,302,lgb_LGBMClassifier,2,"(0, 3, 2)_lgb_LGBMClassifier_date_params.csv",0.670807,"{'boosting_type': 'dart', 'class_weight': None...",9.0,lgb_LGBMClassifier2
186,186,RandomForestClassifier,36,"(0, 0, 36)_RandomForestClassifier_date_params.csv",0.669903,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",10.0,RandomForestClassifier36


In [42]:
final_results = best_model_p_func.merge(results,on=['model','param_config']).sort_values('rank_prod')[['func','model','param_config','precision_prod','rank_prod']]#.params.values

last_fold_results = outputs_metrics.query("time == 9").sort_values('precision_test',ascending=False)
last_fold_results['rank_last_fold']=last_fold_results.precision_test.rank(ascending=False)
#last_fold_results['model'] = last_fold_results['model'].str[:-12]
#pd.merge(last_fold_results.rename({'outputs_metrics':'last_fold_precision'},axis=1),final_results,on=['model','param_config'])
last_fold_results['model'] = last_fold_results['model'].str[:-12] 

In [43]:
last_fold_results

,Unnamed: 0,threshold,precision_test,precision_threshold,filename,model,param_config,time,rank_last_fold
2139,3463,0.374200,0.666667,0.000000,"(9, 6, 20)_SVC_date_params.csv",SVC,20,9,1.0
3499,3349,0.686919,0.627599,0.435789,"(9, 3, 9)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier,9,9,2.0
3229,3322,0.677751,0.624113,0.423313,"(9, 3, 29)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier,29,9,3.0
3269,3326,0.694461,0.617284,0.416499,"(9, 3, 32)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier,32,9,4.0
1599,3159,0.570983,0.615530,0.575000,"(9, 0, 17)_RandomForestClassifier_date_params.csv",RandomForestClassifier,17,9,5.0
3029,3302,0.661851,0.614916,0.423313,"(9, 3, 10)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier,10,9,6.0
3049,3304,0.673211,0.613309,0.479167,"(9, 3, 12)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier,12,9,7.0
3369,3336,0.682864,0.612774,0.405088,"(9, 3, 41)_lgb_LGBMClassifier_date_params.csv",lgb_LGBMClassifier,41,9,8.0
1879,3187,0.637526,0.612524,0.591429,"(9, 0, 42)_RandomForestClassifier_date_params.csv",RandomForestClassifier,42,9,9.0
2609,3360,0.768360,0.612371,0.398844,"(9, 4, 18)_XGBClassifier_date_params.csv",XGBClassifier,18,9,10.0


In [45]:
compare_last_prod = pd.merge(results,last_fold_results,on=['model','param_config']).sort_values('rank_last_fold')[['model','param_config','precision_test','precision_prod','rank_last_fold','rank_prod']]
compare_last_prod['diff_last_prod'] =compare_last_prod['rank_last_fold'] - compare_last_prod.rank_prod
fig = compare_last_prod.plot(kind ='scatter',x='rank_last_fold',y='rank_prod',width=1000,height=1000)
plotly.offline.plot(fig, filename='fig_ranks_last_prod.html')


'fig_ranks_last_prod.html'

In [ ]:
#best_model_p_func['model'] = best_model_p_func['model'].str[:-12]


In [ ]:
final_results = pd.merge(final_results,last_fold_results[['model','param_config','rank_last_fold']],on=['model','param_config'])
final_results

In [ ]:
best_model_last[0][:-12]

In [ ]:
pd.read_csv('outputs_models2/(0, 6, 17)_MLPClassifier.csv').iloc[0].get_p

In [ ]:
best_model_p_func[['model','param_config']]

In [ ]:
best_model_last[0]

In [ ]:
f"model ==' {best_model_last[0]}'"